In [1]:
import catboost as cb
import gc
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sys
sys.path.append("../")
sys.path.append("../../")
import warnings
warnings.simplefilter("ignore")
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, 
    get_cols
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)

In [3]:
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
from colorama import Fore, Back, Style
plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

In [4]:
%load_ext autoreload
%autoreload

### Read Data

In [5]:
%%time
train_agg = read_file(f"../{PROCESSED_DATA_PATH}/v21/train_agg.pkl")

Shape of data: (458913, 3310)
CPU times: user 2.52 s, sys: 9.4 s, total: 11.9 s
Wall time: 20.8 s


In [6]:
labels = read_file(f"../{RAW_DATA_PATH}/train_labels.csv")
target = labels["target"].values

Shape of data: (458913, 2)


In [7]:
%%time
train_agg.drop(columns=NON_FEATURE_COLUMNS, errors="ignore", inplace=True)

CPU times: user 362 ms, sys: 2.56 s, total: 2.92 s
Wall time: 5.75 s


In [8]:
missing = train_agg.isnull().sum()

In [9]:
missing_cols = missing[missing > 0].index.tolist()

In [10]:
%%time
train_agg.drop(columns=missing_cols, errors="ignore", inplace=True)

CPU times: user 274 ms, sys: 2.08 s, total: 2.35 s
Wall time: 4.32 s


In [11]:
cat_features = train_agg.select_dtypes("category").columns.tolist()

In [12]:
train_agg.shape

(458913, 3308)

### MinMaxScaler

In [13]:
m = MinMaxScaler()

In [14]:
train_agg_number = train_agg.select_dtypes(np.number)
train_agg_cat = train_agg.select_dtypes("category")

In [15]:
train_agg_number.shape, train_agg_cat.shape

((458913, 3290), (458913, 18))

In [16]:
train_agg_number = pd.DataFrame(m.fit_transform(train_agg_number), columns=train_agg_number.columns)

In [17]:
train_agg = pd.concat([train_agg_number, train_agg_cat], axis=1)

In [18]:
train_agg.head()

,P_2_last,S_3_last,P_3_last,S_5_last,S_6_last,S_7_last,S_8_last,S_12_last,S_13_last,S_15_last,...,D_126_last,B_30_first,B_38_first,D_63_first,D_64_first,D_68_first,D_114_first,D_117_first,D_120_first,D_126_first
0,0.945736,0.138889,0.588235,0.008571,0.0,0.119497,0.339286,0.337143,0.7,0.1875,...,2,0,0,0,0,0,1,4,0,2
1,0.906977,0.155556,0.567227,0.011429,0.0,0.182390,0.267857,0.340000,0.1,0.1875,...,2,0,0,3,0,0,1,0,0,2
2,0.906977,0.127778,0.588235,0.000000,1.0,0.106918,0.232143,0.340000,0.0,0.1875,...,2,0,1,3,1,0,0,0,0,2
3,0.705426,0.222222,0.609244,0.005714,0.0,0.226415,0.053571,0.300000,0.2,0.3125,...,2,0,0,3,0,4,1,0,0,2
4,0.899225,0.127778,0.567227,0.000000,1.0,0.106918,0.232143,0.337143,0.0,0.1875,...,2,0,0,3,0,0,1,4,0,2


In [19]:
del train_agg_number, train_agg_cat

### Tune Catboost using Optuna (KFold)

In [20]:
# prev_study = joblib.load(f"../catboost_new_exp/optuna_study.pkl")
# tdf = prev_study.trials_dataframe()
# tdf = tdf.loc[~tdf["value"].isnull()].reset_index(drop=True)
# tdf.columns

In [ ]:
class AmexMetric:
    @staticmethod
    def calc(y_true, y_pred):
        metric, _, _ = amex_metric(y_true, y_pred)
        return metric
    
    def is_max_optimal(self):
        return True # greater is better

    def evaluate(self, approxes, target, weight):            
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target).astype(int)
        approx = approxes[0]
        score = self.calc(y_true, approx)
        return score, 1

    def get_final_error(self, error, weight):
        return error

In [22]:
def objective(trial):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=111)

    param = {
        "objective": "Logloss",
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.035, 0.07, log=True),
        "depth": trial.suggest_int("depth", 12, 16),
        "random_strength": trial.suggest_float("random_strength", 1.2, 1.4),
        "class_weights": trial.suggest_categorical("class_weights", [[1, 1.25], [1, 1.35]]),
        "n_estimators": trial.suggest_categorical("n_estimators", [750, 850, 950, 1050]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 6, 15, log=True),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [1500, 2048, 2560]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.04, 0.05, 0.06]),
        "max_bin": trial.suggest_categorical("max_bin", [127, 190, 255]),
        "used_ram_limit": "16gb",
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["MVS", "Bernoulli"]),
        # "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "grow_policy": "Depthwise",
        "eval_metric": AmexMetric(),
    }

    if param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.75, 0.85)
    
    print(param)
    train_score_list, val_score_list = [], []
    for fold, (idx_tr, idx_va) in zip(range(1, 5+1), kf.split(train_agg, target)):
        fold = str(fold)
        X_train, y_train = train_agg.iloc[idx_tr], target[idx_tr]
        X_val, y_val = train_agg.iloc[idx_va], target[idx_va]
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = cb.CatBoostClassifier(**param)
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_val, y_val)], 
                verbose=0, 
                cat_features=cat_features,
                early_stopping_rounds=100
            )
        y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
        train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
        train_data, X_train, y_train = None, None, None
        y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
        val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
        valid_data, X_val, y_val = None, None, None
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        # if val_score > best_scores_json["validation"][fold]:
        #     best_scores_json["train"][fold] = train_score
        #     best_scores_json["validation"][fold] = val_score
        #     with open(f'{CURRENT_EXP_PATH}/best_scores.json', "w") as outfile:
        #         json.dump(best_scores_json, outfile)
        #     joblib.dump(model, f'{CURRENT_EXP_PATH}/models/model{fold}.pkl')
        # elif np.mean(train_score_list) >= np.mean(list(best_scores_json["train"].values())) + 0.02:
        #     print(f"Train score too high (overfitting), start a new trial")
        #     return np.mean(val_score_list)
        joblib.dump(model, f'./models/model{fold}_seed111_{val_score:.5f}.pkl')
        print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
        print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
        print(f"Clear cache {gc.collect()}")
        
    return np.mean(val_score_list)

In [23]:
study = optuna.create_study(direction="maximize")

[I 2022-08-22 15:21:00,072] A new study created in memory with name: no-name-3af66aa1-0221-497c-a301-9039744235ea


In [24]:
study.optimize(objective, n_trials=2)

{'objective': 'Logloss', 'colsample_bylevel': 0.05182713519237734, 'depth': 14, 'random_strength': 1.3195464793304954, 'class_weights': [1, 1.25], 'n_estimators': 1050, 'l2_leaf_reg': 6.571206669694062, 'min_data_in_leaf': 2048, 'learning_rate': 0.04, 'max_bin': 190, 'used_ram_limit': '16gb', 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise', 'eval_metric': <__main__.AmexMetric object at 0x284ab59f0>, 'subsample': 0.8139073993368319}
Fold 1 | Train Score = 0.89415 (0.9610, 0.8273)
Fold 1 | Val Score = 0.79319 (0.9223, 0.6641)
Clear cache 23
Fold 2 | Train Score = 0.85423 (0.9468, 0.7616)
Fold 2 | Val Score = 0.79119 (0.9223, 0.6601)
Clear cache 50
Fold 3 | Train Score = 0.83834 (0.9410, 0.7357)
Fold 3 | Val Score = 0.79048 (0.9214, 0.6596)
Clear cache 50
Fold 4 | Train Score = 0.88294 (0.9573, 0.8086)
Fold 4 | Val Score = 0.79317 (0.9227, 0.6637)
Clear cache 50


[I 2022-08-22 16:05:31,105] Trial 0 finished with value: 0.7919513304837243 and parameters: {'colsample_bylevel': 0.05182713519237734, 'depth': 14, 'random_strength': 1.3195464793304954, 'class_weights': [1, 1.25], 'n_estimators': 1050, 'l2_leaf_reg': 6.571206669694062, 'min_data_in_leaf': 2048, 'learning_rate': 0.04, 'max_bin': 190, 'bootstrap_type': 'Bernoulli', 'subsample': 0.8139073993368319}. Best is trial 0 with value: 0.7919513304837243.


Fold 5 | Train Score = 0.88127 (0.9564, 0.8062)
Fold 5 | Val Score = 0.79172 (0.9231, 0.6603)
Clear cache 50
{'objective': 'Logloss', 'colsample_bylevel': 0.06927487743086469, 'depth': 17, 'random_strength': 1.315655573680709, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 10.703142108935726, 'min_data_in_leaf': 2048, 'learning_rate': 0.04, 'max_bin': 255, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'eval_metric': <__main__.AmexMetric object at 0x1602c8dc0>}


[W 2022-08-22 16:06:31,609] Trial 1 failed because of the following error: CatBoostError('catboost/private/libs/options/oblivious_tree_options.cpp:125: Maximum tree depth is 16')
Traceback (most recent call last):
  File "/Users/wklee/miniconda3/envs/amex/lib/python3.10/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/rx/900hbpv51bsg7d73m0zmd11m0000gn/T/ipykernel_11170/630710937.py", line 34, in objective
    model.fit(
  File "/Users/wklee/miniconda3/envs/amex/lib/python3.10/site-packages/catboost/core.py", line 5007, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/Users/wklee/miniconda3/envs/amex/lib/python3.10/site-packages/catboost/core.py", line 2262, in _fit
    train_params = self._prepare_train_params(
  File "/Users/wklee/miniconda3/envs/amex/lib/python3.10/site-packages/catboost/core.py", line 2194,

CatBoostError: catboost/private/libs/options/oblivious_tree_options.cpp:125: Maximum tree depth is 16

In [87]:
study.optimize(objective, n_trials=2)

{'objective': 'Logloss', 'colsample_bylevel': 0.06277312188025866, 'depth': 16, 'random_strength': 1.3657432521345299, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 7.455236317905622, 'min_data_in_leaf': 1500, 'learning_rate': 0.05, 'max_bin': 127, 'used_ram_limit': '16gb', 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise', 'eval_metric': <__main__.AmexMetric object at 0x28c33cfa0>, 'subsample': 0.7975001435377027}
Fold 1 | Train Score = 0.93253 (0.9737, 0.8913)
Fold 1 | Val Score = 0.78998 (0.9218, 0.6582)
Clear cache 422



KeyboardInterrupt



Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x119131210>> (for post_execute):



KeyboardInterrupt



In [65]:
study.optimize(objective, n_trials=2)

{'objective': 'Logloss', 'colsample_bylevel': 0.040243910747011113, 'depth': 14, 'random_strength': 1.3417270232978231, 'class_weights': [1, 1.35], 'n_estimators': 750, 'l2_leaf_reg': 7.704397279944718, 'min_data_in_leaf': 1500, 'learning_rate': 0.05, 'max_bin': 190, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise', 'eval_metric': <__main__.AmexMetric object at 0x15467bfd0>}
Fold 1 | Train Score = 0.89183 (0.9603, 0.8234)
Fold 1 | Val Score = 0.78925 (0.9217, 0.6568)
Clear cache 776
Fold 2 | Train Score = 0.87058 (0.9528, 0.7883)
Fold 2 | Val Score = 0.78951 (0.9220, 0.6570)
Clear cache 50



KeyboardInterrupt



In [47]:
def objective(trial):
    param = {
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.03, 0.1, log=True),
        "depth": trial.suggest_int("depth", 6, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["MVS"]
        ),
        "n_estimators": trial.suggest_categorical("n_estimators", [1500, 1750, 2000]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2, 5, log=True),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [1024, 2048]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.03, 0.06, 0.1]),
        "max_bin": trial.suggest_categorical("max_bin", [127, 255]),
        "objective": "Logloss",
        "used_ram_limit": "20gb",
        "thread_count": 16,
        "eval_metric": AmexMetric(),
        # "task_type": "GPU",
        "devices": '0:1'
    }

    # if param["bootstrap_type"] == "Bernoulli":
    #     param["subsample"] = trial.suggest_float("subsample", 0.6, 0.85)
    
    print(param)
    train_score_list, val_score_list = [], []
    for fold in range(5):
        X_train = pd.concat([df for idx, df in df_dict.items() if idx != fold], ignore_index=True)
        X_val = df_dict[fold]
        print("X shape: ", X_train.shape, X_val.shape)
        y_train = pd.concat([y_dict[idx] for idx in range(5) if idx != fold], ignore_index=True)
        y_val = y_dict[fold]
        print("Y shape: ", y_train.shape, y_val.shape)
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = cb.CatBoostClassifier(**param)
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_val, y_val)], 
                use_best_model=True,
                metric_period=50,
                verbose_eval=50,
                cat_features=cat_features,
                early_stopping_rounds=100
            )
        y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
        train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
        train_data, X_train, y_train = None, None, None
        y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
        val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
        valid_data, X_val, y_val = None, None, None
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
        print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
        print(f"Clear cache {gc.collect()}")
        
    return np.mean(val_score_list)

In [48]:
# study = joblib.load(f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [49]:
study = optuna.create_study(direction="maximize")

[I 2022-08-21 03:38:15,617] A new study created in memory with name: no-name-97d35944-bcfb-4dea-a2ce-95bdf592cd65


In [50]:
study.optimize(objective, n_trials=50)

{'colsample_bylevel': 0.050454698092565815, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1500, 'l2_leaf_reg': 2.7451289723741215, 'min_data_in_leaf': 2048, 'learning_rate': 0.03, 'max_bin': 63, 'objective': 'Logloss', 'used_ram_limit': '20gb', 'thread_count': 16, 'eval_metric': <__main__.AmexMetric object at 0x7fcbec4a29e0>, 'devices': '0:1'}
X shape:  (330410, 4368) (82603, 4368)
Y shape:  (330410,) (82603,)


0:	learn: 0.6915845	test: 0.6905689	best: 0.6905689 (0)	total: 1.1s	remaining: 27m 33s
50:	learn: 0.7672712	test: 0.7618455	best: 0.7618455 (50)	total: 34.9s	remaining: 16m 31s
100:	learn: 0.7787548	test: 0.7729213	best: 0.7729213 (100)	total: 1m 7s	remaining: 15m 30s
150:	learn: 0.7854660	test: 0.7781240	best: 0.7784973 (147)	total: 1m 39s	remaining: 14m 48s
200:	learn: 0.7894451	test: 0.7814150	best: 0.7817305 (199)	total: 2m 11s	remaining: 14m 12s
250:	learn: 0.7930745	test: 0.7827799	best: 0.7827828 (248)	total: 2m 43s	remaining: 13m 31s
300:	learn: 0.7957016	test: 0.7848815	best: 0.7848815 (300)	total: 3m 13s	remaining: 12m 51s
350:	learn: 0.7984149	test: 0.7849632	best: 0.7856020 (342)	total: 3m 44s	remaining: 12m 15s
400:	learn: 0.8003926	test: 0.7858359	best: 0.7858359 (400)	total: 4m 15s	remaining: 11m 40s
450:	learn: 0.8028003	test: 0.7865907	best: 0.7866292 (447)	total: 4m 46s	remaining: 11m 6s
500:	learn: 0.8056789	test: 0.7872089	best: 0.7872686 (484)	total: 5m 18s	remaini

0:	learn: 0.6889373	test: 0.6837398	best: 0.6837398 (0)	total: 1.13s	remaining: 28m 9s
50:	learn: 0.7670921	test: 0.7651666	best: 0.7651666 (50)	total: 35.6s	remaining: 16m 51s
100:	learn: 0.7785601	test: 0.7742705	best: 0.7742705 (100)	total: 1m 7s	remaining: 15m 33s
150:	learn: 0.7849044	test: 0.7781200	best: 0.7783552 (147)	total: 1m 40s	remaining: 14m 57s



KeyboardInterrupt



In [ ]:
# joblib.dump(study, f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [62]:
study_df = study.trials_dataframe()

In [63]:
study_df = study_df.loc[study_df["state"] == "COMPLETE"]

In [ ]:
# for col in get_cols(study_df, "params"):
#     sns.scatterplot(data=study_df, x=study_df[col], y=study_df["value"])
#     plt.show()

### Single Experiment

In [60]:
param = {
    "colsample_bylevel": 0.1,
    "depth": 10,
    "boosting_type": "Ordered",
    "bootstrap_type": "MVS",
    "n_estimators": 1000,
    "l2_leaf_reg": 1,
    "min_data_in_leaf": 1024,
    "learning_rate": 0.05,
    "max_bin": 255,
    "objective": "Logloss",
    "used_ram_limit": "20gb",
    "thread_count": 16,
    "random_strength": 1.3,
    "scale_pos_weight": 1.35,
    "eval_metric": AmexMetric(),
    # "grow_policy": "Depthwise",
    # "max_leaves": 90,
    # "task_type": "GPU",
}

In [ ]:
print(param)
train_score_list, val_score_list = [], []
for fold in range(5):
    X_train = pd.concat([df for idx, df in df_dict.items() if idx != fold], ignore_index=True)
    X_val = df_dict[fold]
    print("X shape: ", X_train.shape, X_val.shape)
    y_train = pd.concat([y_dict[idx] for idx in range(5) if idx != fold], ignore_index=True)
    y_val = y_dict[fold]
    print("Y shape: ", y_train.shape, y_val.shape)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model = cb.CatBoostClassifier(**param)
        model.fit(
            X_train, 
            y_train, 
            eval_set=[(X_val, y_val)], 
            use_best_model=True,
            metric_period=50,
            verbose_eval=50,
            cat_features=cat_features,
            early_stopping_rounds=100
        )
    y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
    train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
    train_data, X_train, y_train = None, None, None
    y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
    val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
    valid_data, X_val, y_val = None, None, None
    train_score_list.append(train_score)
    val_score_list.append(val_score)
    print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
    print(f"Clear cache {gc.collect()}")

{'colsample_bylevel': 0.1, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1000, 'l2_leaf_reg': 1, 'min_data_in_leaf': 1024, 'learning_rate': 0.05, 'max_bin': 255, 'objective': 'Logloss', 'used_ram_limit': '20gb', 'thread_count': 16, 'random_strength': 1.3, 'scale_pos_weight': 1.35, 'eval_metric': <__main__.AmexMetric object at 0x7fc7efb7fee0>}
X shape:  (330410, 4368) (82603, 4368)
Y shape:  (330410,) (82603,)


0:	learn: 0.6764630	test: 0.6732952	best: 0.6732952 (0)	total: 5.14s	remaining: 1h 25m 39s
50:	learn: 0.7755758	test: 0.7685304	best: 0.7685304 (50)	total: 3m 27s	remaining: 1h 4m 15s
100:	learn: 0.7869917	test: 0.7768175	best: 0.7768175 (100)	total: 6m 42s	remaining: 59m 38s
150:	learn: 0.7927805	test: 0.7816190	best: 0.7816291 (147)	total: 9m 51s	remaining: 55m 24s
200:	learn: 0.7976293	test: 0.7832967	best: 0.7836396 (196)	total: 12m 58s	remaining: 51m 33s
250:	learn: 0.8019940	test: 0.7851387	best: 0.7854065 (248)	total: 16m 8s	remaining: 48m 9s
300:	learn: 0.8066188	test: 0.7859108	best: 0.7860225 (289)	total: 19m 21s	remaining: 44m 57s
350:	learn: 0.8118163	test: 0.7867622	best: 0.7870929 (340)	total: 22m 32s	remaining: 41m 40s
400:	learn: 0.8163097	test: 0.7879584	best: 0.7879584 (400)	total: 25m 41s	remaining: 38m 22s
450:	learn: 0.8201758	test: 0.7882930	best: 0.7885578 (443)	total: 28m 51s	remaining: 35m 7s
500:	learn: 0.8239012	test: 0.7886714	best: 0.7888316 (481)	total: 32

0:	learn: 0.6967810	test: 0.6926070	best: 0.6926070 (0)	total: 5.54s	remaining: 1h 32m 11s
